In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import keras
from keras.layers import Dropout
from keras.layers import Activation
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import Activation, BatchNormalization

In [ ]:
AlexNet = Sequential()
AlexNet.add(Conv2D(filters=96,kernel_size=(3,3),strides=(4,4),input_shape=(32,32,3), activation='relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
AlexNet.add(BatchNormalization())
AlexNet.add(Conv2D(60,(5,5),padding='same',activation='relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
AlexNet.add(BatchNormalization())
AlexNet.add(Conv2D(60,(3,3),padding='same',activation='relu'))
AlexNet.add(BatchNormalization())
AlexNet.add(Conv2D(30,(3,3),padding='same',activation='relu'))
AlexNet.add(BatchNormalization())
AlexNet.add(Conv2D(20,(3,3),padding='same',activation='relu'))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
AlexNet.add(Flatten())
#fully connected layer
AlexNet.add(Dense(512, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(256, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(128, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(64, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(32, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(16, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(8, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(4, activation='relu'))
AlexNet.add(Dropout(0.1))
AlexNet.add(Dense(10, activation='softmax'))

In [ ]:
#compie the model
AlexNet.compile(optimizer='SGD', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
AlexNet.summary()

In [ ]:
(x_train, y_train), (x_test, y_test)  = keras.datasets.cifar10.load_data()
#load the dataset

In [ ]:
# Reshape converting 2D to 1D
y_test = y_test.reshape(-1,)
y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)
y_train = y_train.reshape(-1,)
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
#train the model
history1 = AlexNet.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test),verbose=1)

In [ ]:
#evalute accuracy on testing data
evaluation = AlexNet.evaluate(x_test, y_test)
print('Test Accuracy: {}'.format(evaluation[1]))

In [ ]:
#getting original weights
original_w = AlexNet.get_weights()

In [ ]:
accuracys=[]

In [ ]:
num_layers=9

In [ ]:
import random
import numpy as np
import struct
x=num_layers - 1
new_acc = []
indexes = []
weights = original_w
for i in range(len(original_w)-2,len(original_w)-1-(2*x),-2):
   indexes.append(i)
l1=indexes[::-1]
for i in range(len(l1)):  
  for j in range(50):
      k=random.randint(0,len(weights[l1[i]])-1)
      idx=random.randint(0,len(weights[l1[i]][k])-1)
      r_weight = weights[l1[i]][k][idx]
      # print(r_weight)
      r_weight = format(struct.unpack('!I', struct.pack('!f', r_weight))[0], '032b')
      r_bit_index = random.randint(0,31)
      r_bit = r_weight[r_bit_index]
      
      if(r_bit == '0'):
        r_weight = r_weight[:r_bit_index] + '1' + r_weight[r_bit_index + 1:]
      if(r_bit == '1'):
        r_weight = r_weight[:r_bit_index] + '0' + r_weight[r_bit_index + 1:]
      r_weight = struct.unpack('!f',struct.pack('!I', int(r_weight, 2)))[0]
      # print(r_weight)
      weights[l1[i]][k][idx]=r_weight
  AlexNet.set_weights(weights)
  print("layer ", i+1)
  print("no of weights changed: ",j+1)
  acc = AlexNet.evaluate(x_test, y_test)
  print("test accuracy {}".format(acc[1]))
  new_acc.append(acc[1]*100)
  weights=original_w
  AlexNet.set_weights(original_w)
  print("\n")
accuracys.append(new_acc)

In [ ]:
#here we are changing the weights of each layer by multilplying the scallar value and obtained accuracy on change weeights
accuracys=[]
x=num_layers
for i in range(len(original_w)-1,len(original_w)-1-num_layers,-1):
  accuracy=[]
  print("Layer :",x)
  for scalar in [.25, 0.5, .75, 0.9, 1.1,1.25,1.5,1.75,2.]:
    new_w=[]
    for w in range(len(original_w)):
      if(w==i):
        new_w.append(original_w[w]*scalar)
      else:
        new_w.append(original_w[w])
    AlexNet.set_weights(new_w)
    evaluation = AlexNet.evaluate(x_test, y_test)
    print("Scallar {} --> test accuracy {}".format(scalar,evaluation[1]))
    accuracy.append(evaluation[1])
    AlexNet.set_weights(original_w)
  x-=1
  AlexNet.set_weights(original_w)
  accuracys.append(accuracy)

In [ ]:
average=[]
for i in accuracys:
  average.append(sum(i)/len(i))
print(average)

In [ ]:
scalars=[ .25, 0.5, .75, 0.9, 1.1,1.25,1.5,1.75,2.]

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(accuracys)):
  plt.plot(scalars, accuracys[i], label=f'layer {num_layers-i}')
plt.xlabel('Scalar')
plt.ylabel('Accuracy')
plt.title('Effect on accuracy when weights are multiplied by scalers on alexnet variant')
plt.legend()
plt.show()

In [ ]:
accuracys=[]
x=num_layers
for i in range(len(original_w)-1,len(original_w)-1-num_layers,-1):
  accuracy=[]
  print("Layer :",x)
  for scalar in [.25, 0.5, .75, 0.9, 1.1,1.25,1.5,1.75,2.]:
    new_w=[]
    for w in range(len(original_w)):
      if(w==i):
        new_w.append(original_w[w]+scalar)
      else:
        new_w.append(original_w[w])
    AlexNet.set_weights(new_w)
    evaluation = AlexNet.evaluate(x_test, y_test)
    print("Scallar {} --> test accuracy {}".format(scalar,evaluation[1]))
    accuracy.append(evaluation[1])
    AlexNet.set_weights(original_w)
  x-=1
  AlexNet.set_weights(original_w)
  accuracys.append(accuracy)

In [ ]:
average=[]
for i in accuracys:
  average.append(sum(i)/len(i))
print(average)

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(accuracys)):
  plt.plot(scalars, accuracys[i], label=f'layer {num_layers-i}')
plt.xlabel('Scalar')
plt.ylabel('Accuracy')
plt.title('Effect on accuracy when weights are added by scalers')
plt.legend()
plt.show()

In [ ]:
accuracys=[]

In [ ]:
import random
import numpy as np
import struct
x=num_layers - 1
new_acc = []
indexes = []
weights = original_w.copy()
for i in range(len(original_w)-2,len(original_w)-1-(2*x),-2):
   indexes.append(i)
l1=indexes[::-1]
for i in range(len(l1)):
  
  for j in range(2,3,2):
    for k in range(len(weights[l1[i]])):
      
      idx=random.randint(0,len(weights[l1[i]][k])-1)
      r_weight = weights[l1[i]][k][idx]
      r_weight = format(struct.unpack('!I', struct.pack('!f', r_weight))[0], '032b')
      r_bit_index = random.randint(0,31)
      r_bit = r_weight[r_bit_index]
      
      if(r_bit == '0'):
        r_weight = r_weight[:r_bit_index] + '1' + r_weight[r_bit_index + 1:]
      if(r_bit == '1'):
        r_weight = r_weight[:r_bit_index] + '0' + r_weight[r_bit_index + 1:]
      r_weight = struct.unpack('!f',struct.pack('!I', int(r_weight, 2)))[0]
      weights[l1[i]][k][idx]=r_weight
  model1.set_weights(weights)
  print("no of weights changed: ",j)
  # acc = model1.evaluate(x_test, y_test)
  # print("test accuracy {}".format(acc[1]))
  evaluation = model1.evaluate(x_test, y_test_cat)
  print("test accuracy {}".format(evaluation[1]))
  new_acc.append(evaluation[1]*100)
  weights=original_w
  print("\n")

In [ ]:
acc_graph=[]
for j in range(len(accuracys[0])):
  a=[]
  for i in range(len(accuracys)):
    a.append(accuracys[i][j])
  acc_graph.append(a)

In [ ]:
numberofweightchange=[50,75,100,125,150]
for i in range(len(acc_graph)):
  plt.plot(numberofweightchange, acc_graph[i], label=f'layer {len(acc_graph)-i-1}')
plt.xlabel('Number of weights changed')
plt.ylabel('Accuracy')
plt.title('Effect on accuracy when weights are changed by bit-flip')
plt.legend()
plt.show()

In [ ]:
average=[]
for i in acc_graph:
  average.append(sum(i)/len(i))
print(average)